<a href="https://colab.research.google.com/github/mahu0926/NVIDIA-PilotNet-Implementation/blob/main/NVIDIA_PilotNet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA PilotNet Implementation
The goal of this code is to create an implementation of the popular end-to-end learning system of NVIDIA's PilotNet. 

[End to End Learning for Self Driving Cars](https://arxiv.org/pdf/1604.07316v1.pdf)

[Explaining How a Deep Neural Network Trained with End-to-End Learning Steers a Car](https://arxiv.org/pdf/1704.07911v1.pdf)

[Sully Chen's Autopilot Github](https://github.com/SullyChen/Autopilot-TensorFlow)

Opening the Data



In [ ]:
import zipfile
import os

!pip install gdown
!gdown 1PZWa6H0i1PCH9zuYcIh5Ouk_p-9Gh58B

In [ ]:
!unzip 07012018.zip

In [ ]:
# Importing everything
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
import keras
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential, save_model, load_model
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
import numpy as np
from keras.callbacks import TensorBoard
import time
import datetime
import cv2
import os
import tensorflow.keras
from PIL import Image
print('tf version', tf.__version__)
print('keras version', tf.keras.__version__)
print('gpu is ','available' if tf.config.list_physical_devices('GPU') else 'not available')

In [ ]:

#Reading data.txt - creating x, y, year, time variables
x = []
y = []
year = []
time = []
with open("data.txt") as f:
  for line in f:
    x.append(line.split()[0])
    string = line.split()[1]
    y.append((float(string.split(',')[0]) * 3.14159265 / 180))
    year.append(string.split(',')[1])
    time.append(line.split()[2])

num_images = len(x)
print(num_images)

In [ ]:
# Checking the image
from PIL import Image
filepath = "data/9991.jpg"
Image.open(filepath)

In [ ]:
# Creating Dataframes
df = pd.DataFrame(list(zip(x, y)), columns =['imagepath', 'steering_command'])
df_train = df.sample(frac=0.9)
mask = df.imagepath.isin(df_train.imagepath)
df_test = df[~mask]

In [ ]:
# Visualizing the data
import matplotlib.pyplot as plt
import cv2 as cv
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random

fig = plt.figure(figsize=(20., 20.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(3, 3),
                 axes_pad=0.5,
                 label_mode="L",
                 )

for ax, num in zip(grid, list(range(9))):
    n = random.randint(0,len(df))
    img = cv.imread("data/" + df.imagepath[n])
    #plt.figure(figsize = (10, 25))
    ax.set_title("STEERING COMMAND: " + str(df.steering_command[n]))
    ax.imshow(img)


In [ ]:
# Creating histogram of y values
plt.hist(df['steering_command'], bins=10, log=True)
plt.title("Steering Angle Histogram")
plt.xlabel("Steering Angle Value")
plt.ylabel("Frequency")

In [ ]:
print(df_train)

In [ ]:
# Creating Dataframe Iterators

valsplit = 0.1
BS = 100
height = 455
width = 256

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split = valsplit)
test_datagen = ImageDataGenerator(rescale=1./255.)
train_generator = train_datagen.flow_from_dataframe(
        df_train,
        directory='data/',
        x_col='imagepath',
        y_col='steering_command',
        target_size=(height, width),
        batch_size=BS,
        class_mode='raw',
        subset = 'training'
        )
validation_generator = train_datagen.flow_from_dataframe(
        df_train,
        directory='data/',
        x_col='imagepath',
        y_col='steering_command',
        target_size=(height, width),
        batch_size=BS,
        class_mode='raw',
        subset = 'validation'
        )
test_generator=test_datagen.flow_from_dataframe(
        df_test,
        directory = 'data/',
        x_col='imagepath',
        y_col='steering_command',
        target_size=(height, width),
        batch_size=BS,
        class_mode='raw',
        )

Creating the models

In [ ]:
from keras.models import Sequential, save_model, load_model
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf

depth = 3

inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1

In [ ]:
# Pure PilotNet Paper Implementation 
model = Sequential()
model.add(Conv2D(24, kernel_size=(5, 5), strides=(2, 2), padding="same",input_shape=inputShape, activation="relu"))
model.add(Conv2D(36, kernel_size=(5, 5), strides=(2, 2), padding='same', activation="relu"))
model.add(Conv2D(48, kernel_size=(5, 5), strides=(2, 2), padding='same', activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation="relu"))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1))
model.summary()


In [ ]:
# PilotNet with Dropout
model = Sequential()
model.add(Conv2D(24, kernel_size=(5, 5), strides=(2, 2), padding="same",input_shape=inputShape, activation="relu"))
model.add(Conv2D(36, kernel_size=(5, 5), strides=(2, 2), padding='same', activation="relu"))
model.add(Conv2D(48, kernel_size=(5, 5), strides=(2, 2), padding='same', activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation="relu"))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation="relu"))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(50, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(10, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(1))
model.summary()

In [ ]:
# PilotNet with Batch Normalization
n_classes = len(df.steering_command)
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), strides=(2, 2), padding="same",input_shape=inputShape))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))

model.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same'))
model.add(BatchNormalization(axis=chanDim))
model.add(Activation("relu"))

model.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same'))
model.add(Activation("relu"))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(Activation("relu"))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same'))
model.add(Activation("relu"))
model.add(Flatten())

model.add(Dense(100, activation="relu"))
model.add(Dense(50, activation="relu"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1))

model.summary()

In [ ]:
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.5, min_lr= 1e-6, verbose=1)

model.compile(
    loss='mse',
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    #optimizer = 'adam',
    metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [ ]:
history = model.fit(
        train_generator,
        validation_data=validation_generator,
        steps_per_epoch=((1-valsplit)*len(df_train))//BS,
        validation_steps = ((valsplit)*len(df_train))//BS,
        epochs=15,
        callbacks=[early_stop, rlrop])

In [ ]:
import matplotlib.pyplot as plt

rmse = history.history['root_mean_squared_error']
val_rmse = history.history['val_root_mean_squared_error']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(rmse) + 1)
#Train and validation accuracy
plt.plot(epochs, rmse, 'b', label='Training RMSE')
plt.plot(epochs, val_rmse, 'r', label='Validation RMSE')
plt.title('Training and Validation Root Mean Squared Error (RMSE)')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
print("Evaluate Test Set")
score = model.evaluate(test_generator)
print(score)